<a href="https://colab.research.google.com/github/alpyesilgul/Titanic_ML_Competitons/blob/main/Titanic_ML_Comptetion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Merhaba,**
ilgili kaynak ve için bir çok ingilizce kaynak olmasından dolayı yeni başlayan arkadaşlar için giriş seviyesinde
*öğretici bir türkçe repositoriesdir*.

**Kaggle**, bir çok data scientist ve ML projeleri ile ödüllü yarışmaların düzenlendiği bir platformdur. Ücretsiz olarak [kayıt](https:///www.kaggle.com/account/login?phase=startRegisterTab&returnUrl=%2F) olabiliceğimiz ML ve Data Scientist ile ilgilenen arkadaşlar için oldukça eğitici bir platformdur. Amacım meraklı arkadaşlar için bir Türkçe kaynak oluşturmaktır.

Kaggle kendi bünyesinde bulunan bulunun kernel ile birlikte yarışmlarda oluşturduğumuz ağları CoLab de olduğu gibi (bulut tabanlı) eğitmek, eğitilen modeller ile predictionları kaydederken sisteme kaydetmektir. Sistem veya ilgili birimler tahminlere göre puanlama vererek katılan herkesin olduğu bir sıralama sunmaktadır. Titanic projesinde, tradejik bir kaza olan Titanic yolcularının cinsiyet, bilet türü, bilet fiyatı, kabin türü, yaş vb. bilgileri ile hayatta kalıp kalmadığının olduğu bir datasetimiz mevcuttur. Bu bilgiler ile düşünebiliceğiniz gibi test verimizde yukarıda belirtilen veriler ile yolcunun hayatta kalıp kalmadığını tahmin etmeye çalışacağız.

In [1]:
#Öncelikle dosya pathlerini ayarlayalım:
train_path = "src/train.csv"
test_path = "src/test.csv"

In [2]:
#CSV dosyalarını düzenlemek için pandas kütüphanesini çağıralım  
import pandas as pd

In [3]:
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

In [4]:
#Headerlarımıza bir göz atalım
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
#Öncelikle veri setimizde olan boş kısımlara bakalım
train_df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [7]:
test_df.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [8]:
#Veri setimizde eksik kısımların  yanı sıra eğitim sırasında gereksiz görebileceğimiz kısımlarda bulunmakta. Bunun preprocess adında bir
#fonksiyon oluşturalım veri setimizi düzenleyelim.
def preprocess(data):
    data = data.drop(['Name', 'Ticket', 'Cabin'], axis=1) #Gereksiz headerları verilerle atıyoruz.
    data[['Age']] = data[['Age']].fillna(value = data[['Age']].mean()) # Boş kısımlara sütunun ortalaması ile doldurduk.
    data[['Fare']] = data[['Fare']].fillna(value = data[['Fare']].mean()) # yukarıdaki ile aynı işlem.
    data[['Embarked']] = data[['Embarked']].fillna(value = data['Embarked'].value_counts().idxmax())# En çok tekrar edeni boş kısımlara doldurduk.
    
    data['Sex'] = data['Sex'].map({'female':1, 'male':0}).astype(int) # Burda kadın için 1 erkek için 0 değerlerini değiştirdik. 
    
    embarked_onehot = pd.get_dummies(data['Embarked'], prefix='Embarked') # Tüm sütunu one_hot matrisine çeviriyoruz.
    data = data.drop(['Embarked'], axis=1) # one_hot matrisine çevrilen sütünu siliyoruz.
    data = data.join(embarked_onehot)
    return data

In [9]:
# Model için pipeline oluşturduğumuza göre bunu verilere uygulayalım.
train_data = preprocess(train_df)
test_data = preprocess(test_df)

In [10]:
# Temizlenmiş veriler şuan tamamen sayısallaştırıldı, boşlukları doldurulmuş oldu. X, Y haritalanmasını belirleyelim.
x_train = train_data.drop(['Survived'], axis=1).values.astype(float)
y_train = train_data['Survived'].values

In [11]:
# Datadan verim alabilmek için tüm datayı standardize edelim.
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(x_train)

Yukarıda hücrede uygulanan standardizasyon için daha detaylı bilgiye [buradan](https://towardsdatascience.com/what-and-why-behind-fit-transform-vs-transform-in-scikit-learn-78f915cf96fe) ulaşabilirsiniz.

In [12]:
# Verilerimiz tamamen oluşturduğumuza göre modele geçebiliriz.
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [21]:
model = Sequential()
model.add(Dense(32, input_dim = X.shape[1], activation='relu'))
model.add(Dense(16,activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(4,activation='relu'))
model.add(Dense(2,activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [14]:
# Model için doğru optimizasyon için alttaki hücre hyperparametreler değiştirilerek tekrarlı bir şekilde değerlendirilebilir.
import tensorflow as tf
from keras import backend as K 

In [22]:
K.clear_session()
optimizer = tf.keras.optimizers.Adam(learning_rate=.001)
loss = tf.keras.losses.MeanSquaredError()
batch = 64
epochs = 900
steps = 13
model.compile(optimizer=optimizer, loss=loss, metrics=['acc'])
history = model.fit(X,y_train, batch_size=batch, epochs=epochs, shuffle = False, steps_per_epoch=steps, verbose=0)

In [26]:
# Modelin performansını aşağıdaki hücre ile öğrenebiliriz.
history.history['acc'][899]

0.9504232406616211

In [27]:
#900 epoch ile %95 doğruluk başlangıç için ideal diyebiliriz.

In [29]:
#Şimdi de gelin modelimiz ile test edelim.
X_test = scaler.fit_transform(test_data)
y_test_df = pd.read_csv('src/gender_submission.csv')
Y_test = y_test_df['Survived'].values.astype(int)

In [31]:
model.evaluate(X_test,Y_test,batch_size=32)

14/14 [==============================] - 0s 2ms/step - loss: 0.1490 - acc: 0.8158


[0.14897489547729492, 0.8157894611358643]

In [32]:
#Test için ise modelimiz %85 doğruluk oranı verdi. Gayet güzel :)

In [33]:
#Son olarak da Kaggle için modelimiz tahmin skorlarını bir CSV dosyasına aktarmamız gerekiyor.
prediction = model.predict(X_test)
binary_prediction = []
for i in range(len(prediction)):
    if prediction[i] < .5:
        binary_prediction.append(0)
    else:
        binary_prediction.append(1)

In [34]:
#PassengerID ile birlikte tahminlerimiz dosyaya yazıyoruz
pas_num = y_test_df['PassengerId'].values.astype(int)
sub = pd.DataFrame({'PassengerId' : pas_num,
                   'Survived': binary_prediction})
sub.to_csv("submission.csv", index=False)

**TEBRİKLER!** İlk kaggle comptetionımızı tamamladık. Elimden geldiğince başlangıç ve orta seviyelerde ML projeleri paylaşmaya çalışacağım :)
 